In [1]:
import torch

In [2]:
a = torch.tensor([1, 2])
b = torch.tensor([3, 4])
c = torch.tensor([3, 4])


In [3]:
torch.stack((a, b, c), dim=1)

tensor([[1, 3, 3],
        [2, 4, 4]])

In [5]:
a = torch.tensor([[1], [2]])
b = torch.tensor([[1], [2]])

torch.cat([a, b], dim=1)

tensor([[1, 1],
        [2, 2]])

In [1]:
alpha = [1.0] * 5  # Example class weights


In [2]:
alpha

[1.0, 1.0, 1.0, 1.0, 1.0]

In [3]:
import torch
a = [1, 2, torch.tensor(3)]
torch.tensor(a)

tensor([1, 2, 3])

In [1]:
import cv2

if not cv2.cuda.getCudaEnabledDeviceCount():
    print("CUDA-enabled GPU not found. Exiting...")

CUDA-enabled GPU not found. Exiting...


In [2]:
from sklearn.metrics import precision_recall_fscore_support
import numpy as np


In [6]:
y_pred = np.array([1, 1, 2, 0, 3, 4, 2])
targets = np.array([1, 0, 2, 1, 4, 4, 3])

precision_recall_fscore_support(targets, y_pred, average='macro')


(0.4, 0.4, 0.3666666666666666, None)

In [8]:
y_pred = np.array([1, 0, 2, 1, 4, 4, 2])
targets = np.array([1, 0, 2, 1, 4, 4, 3])

precision_recall_fscore_support(targets, y_pred, average='macro')

c:\Users\User\.conda\envs\EnvMasoudi\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.7, 0.8, 0.7333333333333333, None)